In [1]:
## -- main lib
import pandas as pd
import os
from joblib import dump
## -- sklearn data processing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import PCA
## -- sklearn modeling
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
## -- sklearn metrics
from sklearn.metrics import r2_score

In [2]:
def impute_outliers(data, low, high, replacement):
    return data.apply(lambda x : replacement if x<low or x>high else x).values.reshape(-1,1)

In [3]:
outlier_imputer_num = ColumnTransformer(
    transformers=[
            ('outlier_replacer_size_m2', 
            FunctionTransformer(impute_outliers, kw_args={'low': 10, 'high': 500, 'replacement': 140}),
            'size_m2'),
            ('outlier_replacer_num_bedrooms',
            FunctionTransformer(impute_outliers, kw_args={'low': 0, 'high': 10, 'replacement': 3}),
	        'num_bedrooms'),
            ('outlier_replacer_num_bathrooms', 
            FunctionTransformer(impute_outliers, kw_args={'low': 0, 'high': 5, 'replacement': 2}),
            'num_bathrooms')
    ],
    # Garde les colonnes restantes telles quelles
    remainder='passthrough'
)
outlier_imputer_num

ColumnTransformer(remainder='passthrough',
                  transformers=[('outlier_replacer_size_m2',
                                 FunctionTransformer(func=<function impute_outliers at 0x0000017A9402D2D0>,
                                                     kw_args={'high': 500,
                                                              'low': 10,
                                                              'replacement': 140}),
                                 'size_m2'),
                                ('outlier_replacer_num_bedrooms',
                                 FunctionTransformer(func=<function impute_outliers at 0x0000017A9402D2D0>,
                                                     kw_args={'high': 10,
                                                              'low': 0,
                                                              'replacement': 3}),
                                 'num_bedrooms'),
                                ('outlier_replacer_num_bathrooms',
                                 FunctionTransformer(func=<function impute_outliers at 0x0000017A9402D2D0>,
                                                     kw_args={'high': 5,
                                                              'low': 0,
                                                              'replacement': 2}),
                                 'num_bathrooms')])

In [4]:
# Définir une Fonction personnalisée pour extraire l'année à partir de la date
def extract_year(data):
    # Convertir le type des valeurs de la colonne data en type datetime
    data = pd.to_datetime(data)
    # Extraire l'année de la date
    annee = data.dt.year
    # Transformer la colonne annee (de type Series) en un array numpy 2D
    return annee.values.reshape(-1, 1) 

In [5]:
# Instancier le transformateur personnalisé
year_extractor = FunctionTransformer(extract_year)
year_extractor

FunctionTransformer(func=<function extract_year at 0x0000017AF1BA13F0>)

In [6]:
def to_dataframe_with_columns(data, columns):
    # Retourner un dataframe
    return pd.DataFrame(data, columns=columns)

In [7]:
cols_num = ['size_m2', 'num_bedrooms', 'num_bathrooms', 'distance_school','public_transport_access', 'property_tax']
preprocessor_num = Pipeline(
    steps=[
        ('nan_imputer_num', SimpleImputer(strategy='mean')), # Imputation des NaN
        ('to_dataframe', FunctionTransformer(to_dataframe_with_columns, kw_args={'columns': cols_num})),
        ('outlier_imputer_num', outlier_imputer_num)
])


In [8]:
preprocessor = ColumnTransformer(
    transformers=[
            # Imputation pour les colonnes numériques
            ('num', preprocessor_num, cols_num), 
            # Encodage pour les colonnes catégorielles
            ('cat', OneHotEncoder(), ['city']),
             ('date', year_extractor, 'date_built')
    ]
)
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('nan_imputer_num',
                                                  SimpleImputer()),
                                                 ('to_dataframe',
                                                  FunctionTransformer(func=<function to_dataframe_with_columns at 0x0000017A94054280>,
                                                                      kw_args={'columns': ['size_m2',
                                                                                           'num_bedrooms',
                                                                                           'num_bathrooms',
                                                                                           'distance_school',
                                                                                           'public_transport_access',
                                                                                           'property_tax']})),
                                                 ('outlier_imputer_num',
                                                  ColumnTransform...
                                                                                   FunctionTransformer(func=<function impute_outliers at 0x0000017A9402D2D0>,
                                                                                                       kw_args={'high': 5,
                                                                                                                'low': 0,
                                                                                                                'replacement': 2}),
                                                                                   'num_bathrooms')]))]),
                                 ['size_m2', 'num_bedrooms', 'num_bathrooms',
                                  'distance_school', 'public_transport_access',
                                  'property_tax']),
                                ('cat', OneHotEncoder(), ['city']),
                                ('date',
                                 FunctionTransformer(func=<function extract_year at 0x0000017AF1BA13F0>),
                                 'date_built')])

In [9]:
pipeline = Pipeline(steps=[
    # Imputation des valeurs manquantes
    ('preprocessor', preprocessor),
    # Normalisation des données
    ('scaler', StandardScaler()),
    # reduction dim
    ('pca', PCA()),
    # Modèle de régression
    ('regressor', LinearRegression())
])
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('nan_imputer_num',
                                                                   SimpleImputer()),
                                                                  ('to_dataframe',
                                                                   FunctionTransformer(func=<function to_dataframe_with_columns at 0x0000017A94054280>,
                                                                                       kw_args={'columns': ['size_m2',
                                                                                                            'num_bedrooms',
                                                                                                            'num_bathrooms',
                                                                                                            'distance_school',
                                                                                                            'public_transport_access',
                                                                                                            'property_tax']})),
                                                                  ('outl...
                                                                                                                                 'low': 0,
                                                                                                                                 'replacement': 2}),
                                                                                                    'num_bathrooms')]))]),
                                                  ['size_m2', 'num_bedrooms',
                                                   'num_bathrooms',
                                                   'distance_school',
                                                   'public_transport_access',
                                                   'property_tax']),
                                                 ('cat', OneHotEncoder(),
                                                  ['city']),
                                                 ('date',
                                                  FunctionTransformer(func=<function extract_year at 0x0000017AF1BA13F0>),
                                                  'date_built')])),
                ('scaler', StandardScaler()), ('pca', PCA()),
                ('regressor', LinearRegression())])

In [11]:
df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'synthetic_housing_dataset.csv'))
X = df.drop('price',axis=1)
y = df['price']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7, random_state=45)

In [13]:
param_grid = {
    'pca__n_components': [2, 3 , 4, 5, 6],
}

In [14]:
grid_search = GridSearchCV( 
	estimator = pipeline, 	
	param_grid = param_grid, 
	scoring = 'r2', 
	cv = 5
)
grid_search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('nan_imputer_num',
                                                                                          SimpleImputer()),
                                                                                         ('to_dataframe',
                                                                                          FunctionTransformer(func=<function to_dataframe_with_columns at 0x0000017A94054280>,
                                                                                                              kw_args={'columns': ['size_m2',
                                                                                                                                   'num_bedrooms',
                                                                                                                                   'num_bathrooms',
                                                                                                                                   'distance_school',
                                                                                                                                   'public_transport_acces...
                                                                          'num_bedrooms',
                                                                          'num_bathrooms',
                                                                          'distance_school',
                                                                          'public_transport_access',
                                                                          'property_tax']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['city']),
                                                                        ('date',
                                                                         FunctionTransformer(func=<function extract_year at 0x0000017AF1BA13F0>),
                                                                         'date_built')])),
                                       ('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('regressor', LinearRegression())]),
             param_grid={'pca__n_components': [2, 3, 4, 5, 6]}, scoring='r2')

In [15]:
# Entraîner le pipeline
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('nan_imputer_num',
                                                                                          SimpleImputer()),
                                                                                         ('to_dataframe',
                                                                                          FunctionTransformer(func=<function to_dataframe_with_columns at 0x0000017A94054280>,
                                                                                                              kw_args={'columns': ['size_m2',
                                                                                                                                   'num_bedrooms',
                                                                                                                                   'num_bathrooms',
                                                                                                                                   'distance_school',
                                                                                                                                   'public_transport_acces...
                                                                          'num_bedrooms',
                                                                          'num_bathrooms',
                                                                          'distance_school',
                                                                          'public_transport_access',
                                                                          'property_tax']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['city']),
                                                                        ('date',
                                                                         FunctionTransformer(func=<function extract_year at 0x0000017AF1BA13F0>),
                                                                         'date_built')])),
                                       ('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('regressor', LinearRegression())]),
             param_grid={'pca__n_components': [2, 3, 4, 5, 6]}, scoring='r2')

In [16]:
best_model = grid_search.best_estimator_
best_model.score(X_test, y_test)

0.9690282815796802

In [17]:
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur score R² :", grid_search.best_score_)

Meilleurs paramètres : {'pca__n_components': 6}
Meilleur score R² : 0.9396550885112053


In [18]:
cv_results = pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score')
print(cv_results[['params', 'mean_test_score', 'std_test_score','rank_test_score']])

                     params  mean_test_score  std_test_score  rank_test_score
4  {'pca__n_components': 6}         0.939655        0.025929                1
3  {'pca__n_components': 5}         0.481136        0.113584                2
2  {'pca__n_components': 4}         0.389453        0.153560                3
1  {'pca__n_components': 3}         0.226323        0.187754                4
0  {'pca__n_components': 2}         0.186550        0.106993                5


In [19]:
train_score = grid_search.score(X_train, y_train)
print("Score R² sur train :", train_score)
test_score = grid_search.score(X_test, y_test)
print("Score R² sur test :", test_score)

Score R² sur train : 0.9564505346740458
Score R² sur test : 0.9690282815796802


In [20]:
dump(best_model, os.path.join(os.getcwd(), 'models', "best_pipeline.pkl"))

['c:\\Users\\elies\\OneDrive - De Vinci\\Bureau\\ESILV\\A5\\Tunis\\MLOps\\MLOPS_TD2\\mlops\\models\\best_pipeline.pkl']

In [21]:
# Meilleur pipeline
best_pipeline = grid_search.best_estimator_
# Évaluer sur les données de train
train_score = best_pipeline.score(X_train, y_train)
# Évaluer sur les données de test
test_score = best_pipeline.score(X_test, y_test)

In [22]:
with open('metrics.txt','w') as f :
    f.write(f"Score R2 sur train : {train_score:.2f}\n")
    f.write(f"Score R2 sur test : {test_score:.2f}\n")